In [1]:
import os
import pandas as pd

import ssl
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

import ywc_email_template

In [12]:
letters = pd.read_excel("camper-accaptance-letters/รายชื่อหนังสือรับรอง camper.xlsx")
letters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   twa_ref          23 non-null     object
 1   camper_title     23 non-null     object
 2   camper_fullname  23 non-null     object
 3   camper_major     23 non-null     object
 4   camper_desc      23 non-null     object
 5   recipient        23 non-null     object
 6   to_email         23 non-null     object
 7   cc_emails        2 non-null      object
dtypes: object(8)
memory usage: 1.6+ KB


In [13]:
letters["twa_ref"].apply(lambda x: "สผวท. " + x.split(" ")[1].replace("/","-") + ".pdf")

0     สผวท. 035-66.pdf
1     สผวท. 036-66.pdf
2     สผวท. 037-66.pdf
3     สผวท. 038-66.pdf
4     สผวท. 039-66.pdf
5     สผวท. 040-66.pdf
6     สผวท. 041-66.pdf
7     สผวท. 042-66.pdf
8     สผวท. 043-66.pdf
9     สผวท. 044-66.pdf
10    สผวท. 045-66.pdf
11    สผวท. 046-66.pdf
12    สผวท. 047-66.pdf
13    สผวท. 048-66.pdf
14    สผวท. 049-66.pdf
15    สผวท. 050-66.pdf
16    สผวท. 051-66.pdf
17    สผวท. 052-66.pdf
18    สผวท. 053-66.pdf
19    สผวท. 054-66.pdf
20    สผวท. 055-66.pdf
21    สผวท. 056-66.pdf
22    สผวท. 057-66.pdf
Name: twa_ref, dtype: object

In [14]:
letters["filename"] = letters["twa_ref"].apply(
    lambda x: "สผวท. " + x.split(" ")[1].replace("/", "-") + ".pdf"
)

In [27]:
with open("camper-acceptance-letters/เอกสารโครงการ Young Webmaster Camp ครั้งที่ 19.pdf", "rb") as file:
    ywc19_pdf_file = MIMEApplication(file.read())
    ywc19_pdf_file.add_header(
        "Content-Disposition",
        "attachment",
        filename="เอกสารโครงการ Young Webmaster Camp ครั้งที่ 19.pdf",
    )

In [11]:
import getpass

sender_email = "ywc@webmaster.or.th"
sender_password = getpass.getpass("Password: ")

In [31]:
context = ssl.create_default_context()

with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as smtp:
    smtp.login(sender_email, sender_password)

    for letter in list(letters.itertuples()):
        msg = MIMEMultipart()

        msg["From"] = sender_email
        msg["To"] = letter.to_email

        recipients = list()
        recipients.append(letter.to_email)
        if not pd.isnull(letter.cc_emails):
            recipients += letter.cc_emails.split(",")
            msg["Cc"] = letter.cc_emails

        msg["Subject"] = (
            "[YWC19] ส่งเอกสารรับรองการคัดเลือกเข้าร่วมโครงการ - "
            + letter.camper_fullname
        )

        with open(f"camper-acceptance-letters/letters/{letter.filename}", "rb") as file:
            letter_pdf_file = MIMEApplication(file.read())
            letter_pdf_file.add_header(
                "Content-Disposition",
                "attachment",
                filename=f"หนังสือรับรองการคัดเลือกเข้าร่วมโครงการ YWC19 - {letter.camper_fullname}.pdf",
            )
            
        
        msgText = MIMEText(
            ywc_email_template.build_camper_acceptance_letter_email(
                camper_title=letter.camper_title,
                camper_fullname=letter.camper_fullname,
                camper_major=letter.camper_major,
            ),
            "html",
        )
        msg.attach(msgText)
        msg.attach(letter_pdf_file)
        msg.attach(ywc19_pdf_file)
        
        smtp.sendmail(
            sender_email,
            recipients,
            msg.as_string(),
        )
        print(f"Successfully sent {letter.filename}")

Successfully sent สผวท. 035-66.pdf
Successfully sent สผวท. 036-66.pdf
Successfully sent สผวท. 037-66.pdf
Successfully sent สผวท. 038-66.pdf
Successfully sent สผวท. 039-66.pdf
Successfully sent สผวท. 040-66.pdf
Successfully sent สผวท. 041-66.pdf
Successfully sent สผวท. 042-66.pdf
Successfully sent สผวท. 043-66.pdf
Successfully sent สผวท. 044-66.pdf
Successfully sent สผวท. 045-66.pdf
Successfully sent สผวท. 046-66.pdf
Successfully sent สผวท. 047-66.pdf
Successfully sent สผวท. 048-66.pdf
Successfully sent สผวท. 049-66.pdf
Successfully sent สผวท. 050-66.pdf
Successfully sent สผวท. 051-66.pdf
Successfully sent สผวท. 052-66.pdf
Successfully sent สผวท. 053-66.pdf
Successfully sent สผวท. 054-66.pdf
Successfully sent สผวท. 055-66.pdf
Successfully sent สผวท. 056-66.pdf
Successfully sent สผวท. 057-66.pdf
